# 🗄️ 채팅 기록을 외부 저장소에 저장하기 (Redis)

---

## 📦 Redis 설치 및 도구

> **💡 Redis 설치 안내**  
> 
> ### 🪟 Windows용 Redis 설치
> Windows에서 Redis를 설치하려면 다음 링크를 방문하세요:  
> [Windows용 Redis 설치 프로그램 다운로드](https://github.com/microsoftarchive/redis/releases)
>
> ### 🎨 GUI 관리 도구
> 테스트를 위해 Redis 데이터를 시각화하고 관리하려면 **Another Redis Desktop Manager** 를 사용할 수도 있습니다:  
> [Another Redis Desktop Manager](https://github.com/qishibo/AnotherRedisDesktopManager?tab=readme-ov-file)

---

## 📖 개요

> **학습 목표**  
> 사용자 지정 `ChatMessageStore`를 구현하고 `ChatAgent`와 함께 사용하여 에이전트 채팅 기록을 외부 저장소에 저장하는 방법을 배웁니다.

### 🎯 왜 외부 저장소가 필요한가요?

기본적으로 `ChatAgent`를 사용할 경우, 채팅 기록은:

- **메모리 저장** - `AgentThread` 객체의 메모리에 저장
- **서비스 내부 저장** - 지원될 경우, 추론 서비스 내부에 저장

하지만 다음과 같은 경우 **사용자 지정 저장소**가 필요합니다:

- ❌ 서비스 내부에 채팅 기록을 저장할 수 없는 경우
- ✅ 채팅 기록을 **영구 저장**해야 하는 경우

---

## 🔧 사용자 지정 ChatMessageStore 구현하기

> **구현 요구사항**  
> 사용자 정의 `ChatMessageStore`를 생성하려면 `ChatMessageStore` 프로토콜을 구현하고 필요한 메서드를 제공해야 합니다.

### 📋 필수 구현 메서드

| 메서드 | 역할 | 설명 |
|--------|------|------|
| **add_messages** | 저장 | 저장소에 새 메시지를 추가 |
| **list_messages** | 조회 | 저장소에서 메시지를 가져오기 |

> **⚠️ 중요 사항**  
> `list_messages` 메서드는 메시지를 **시간 순서대로 오름차순**으로 반환해야 합니다.

### 💡 메시지 반환 시 고려사항

반환된 모든 메시지는 내부 채팅 클라이언트를 호출할 때 `ChatAgent`에 의해 사용됩니다. 따라서:

- **모델 한계 고려** - 모델이 처리할 수 있는 만큼의 메시지만 반환
- **기록 축소 로직** - 요약이나 트리밍은 `list_messages`에서 메시지를 반환하기 전에 수행


---

## 💻 ChatMessageStore 구현 예시

> **Redis Lists를 활용한 구현**  
> 다음 샘플은 Redis의 Lists 데이터 구조를 사용하여 채팅 메시지를 저장합니다.

### 🔍 주요 동작 방식

| 동작 | Redis 명령어 | 설명 |
|------|-------------|------|
| **메시지 추가** | `RPUSH` | 시간 순서대로 목록 끝에 추가 |
| **메시지 조회** | `LRANGE` | 시간 순서대로 오름차순 반환 |

### 🔑 키 생성 및 관리

1. **첫 메시지 추가 시** - 해당 스레드에 대한 고유 키 생성
2. **이후 호출** - 생성된 키로 Redis의 채팅 기록 식별

### 💾 상태 직렬화

고유 키 및 설정 정보는 `serialize_state`와 `deserialize_state` 메서드를 통해 `AgentThread`의 일부로 저장됩니다.  
스레드가 생성되거나 재개될 때 `ChatMessageStore` 인스턴스를 생성하고 `AgentThread`에 연결합니다.  
그러면, 스레드 재개 시에 동일한 채팅 기록으로 대화를 이어가는 것이 가능합니다.  

| 메서드 | 역할 |
|--------|------|
| **serialize_state** | 저장소 상태를 저장 |
| **deserialize_state** | 저장소 상태를 복원 |


In [1]:
from collections.abc import Sequence
from typing import Any
from uuid import uuid4
from pydantic import BaseModel
import json
import redis.asyncio as redis
from agent_framework import ChatMessage

class RedisStoreState(BaseModel):
    """Redis 채팅 메시지 저장 데이터를 직렬화 및 역직렬화하기 위한 상태 모델"""

    thread_id: str
    redis_url: str | None = None
    key_prefix: str = "chat_messages"
    max_messages: int | None = None


class RedisChatMessageStore:
    """Redis 기반의 ChatMessageStore 구현체로, Redis Lists를 사용합니다."""

    def __init__(
        self,
        redis_url: str | None = None,
        thread_id: str | None = None,
        key_prefix: str = "chat_messages",
        max_messages: int | None = None,
    ) -> None:
        """Redis 채팅 메시지 저장소를 초기화합니다.

        Args:
            redis_url: Redis 연결 URL (예: "redis://localhost:6379").
            thread_id: 이 대화 스레드에 대한 고유 식별자입니다.
                      제공되지 않으면 UUID가 자동 생성됩니다.
            key_prefix: Redis 키에 대한 접두사로, 서로 다른 애플리케이션을 네임스페이스화합니다.
            max_messages: Redis에 보관할 최대 메시지 수입니다.
                         초과 시, 가장 오래된 메시지가 자동으로 삭제됩니다.
        """
        if redis_url is None:
            raise ValueError("redis_url은 Redis 연결에 필요한 필수정보입니다")

        self.redis_url = redis_url
        self.thread_id = thread_id or f"thread_{uuid4()}"
        self.key_prefix = key_prefix
        self.max_messages = max_messages

        # Redis 클라이언트를 초기화합니다.
        self._redis_client = redis.from_url(redis_url, decode_responses=True)

    @property
    def redis_key(self) -> str:
        """이 스레드의 메시지에 대한 Redis 키를 가져옵니다."""
        return f"{self.key_prefix}:{self.thread_id}"

    async def add_messages(self, messages: Sequence[ChatMessage]) -> None:
        """Redis 저장소에 메시지를 추가합니다.
        Args:
            messages: 저장소에 추가할 ChatMessage 객체들의 시퀀스입니다.
        """
        if not messages:
            return

        # 메세지를 직렬화하고 Redis 리스트에 추가합니다.
        serialized_messages = [self._serialize_message(msg) for msg in messages]
        await self._redis_client.rpush(self.redis_key, *serialized_messages)

        # 설정된 경우, 메시지 제한을 적용합니다.
        if self.max_messages is not None:
            current_count = await self._redis_client.llen(self.redis_key)
            if current_count > self.max_messages:
                # LTRIM을 사용해서 가장 최근의 max_messages만 유지합니다.
                await self._redis_client.ltrim(self.redis_key, -self.max_messages, -1)

    async def list_messages(self) -> list[ChatMessage]:
        """저장소에서 모든 메시지를 시간 순서대로 가져옵니다.

        Returns:
            시간 순서대로 정렬된 ChatMessage 객체들의 리스트 (가장 오래된 것부터).
        """
        # Redis 리스트에서 모든 메시지를 가져옵니다 (가장 오래된 것부터).
        redis_messages = await self._redis_client.lrange(self.redis_key, 0, -1)

        messages = []
        for serialized_message in redis_messages:
            message = self._deserialize_message(serialized_message)
            messages.append(message)

        return messages

    async def serialize_state(self, **kwargs: Any) -> Any:
        """영속성을 위해 현재 저장소 상태를 직렬화합니다.

        Returns:
            직렬화된 저장소 구성을 포함하는 딕셔너리입니다            
        """
        state = RedisStoreState(
            thread_id=self.thread_id,
            redis_url=self.redis_url,
            key_prefix=self.key_prefix,
            max_messages=self.max_messages,
        )
        return state.model_dump(**kwargs)

    async def deserialize_state(self, serialized_store_state: Any, **kwargs: Any) -> None:
        """이 저장소 인스턴스에 상태 데이터를 역직렬화합니다.          

        Args:
            serialized_store_state: 이전에 직렬화된 상태 데이터입니다.
            **kwargs: 역직렬화를 위한 추가 인수입니다.
        """
        if serialized_store_state:
            state = RedisStoreState.model_validate(serialized_store_state, **kwargs)
            self.thread_id = state.thread_id
            self.key_prefix = state.key_prefix
            self.max_messages = state.max_messages

            # Recreate Redis client if the URL changed
            if state.redis_url and state.redis_url != self.redis_url:
                self.redis_url = state.redis_url
                self._redis_client = redis.from_url(self.redis_url, decode_responses=True)

    def _serialize_message(self, message: ChatMessage) -> str:
        """ChatMessage를 JSON 문자열로 직렬화합니다."""
        # Convert ChatMessage to dict using vars() or __dict__
        if hasattr(message, 'model_dump'):
            message_dict = message.model_dump()
        elif hasattr(message, 'to_dict'):
            message_dict = message.to_dict()
        else:
            # Fallback: use __dict__ or vars()
            message_dict = vars(message) if hasattr(message, '__dict__') else dict(message)
        return json.dumps(message_dict, separators=(",", ":"), default=str)

    def _deserialize_message(self, serialized_message: str) -> ChatMessage:
        """JSON 문자열을 ChatMessage로 역직렬화합니다."""
        message_dict = json.loads(serialized_message)
        # Try different deserialization methods
        if hasattr(ChatMessage, 'model_validate'):
            return ChatMessage.model_validate(message_dict)
        elif hasattr(ChatMessage, 'from_dict'):
            return ChatMessage.from_dict(message_dict)
        else:
            # Fallback: use constructor with **kwargs
            return ChatMessage(**message_dict)

    async def clear(self) -> None:
        """저장소에서 모든 메시지를 제거합니다."""
        await self._redis_client.delete(self.redis_key)

    async def aclose(self) -> None:
        """Redis 연결을 종료합니다."""
        await self._redis_client.aclose()

---

## 🚀 ChatAgent에서 사용자 지정 ChatMessageStore 사용하기

### 📌 설정 방법

사용자 지정 `ChatMessageStore`를 사용하려면 에이전트 생성 시 `chat_message_store_factory`를 제공합니다.

> **💡 팩토리 패턴**  
> 이 팩토리를 통해 에이전트는 각 스레드에 대해 `ChatMessageStore`의 새 인스턴스를 생성할 수 있습니다.

In [ ]:
from azure.identity import AzureCliCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient

# 사용자 정의 메시지 저장 팩토리, RedisChatMessageStore를 사용하는 에이전트 생성
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Joker",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    chat_message_store_factory=lambda: RedisChatMessageStore(
        redis_url="redis://localhost:6379"
    )
)

# 영속적인 채팅 기록을 사용하는 에이전트
thread = agent.get_new_thread()
response = await agent.run("미남에 대한 짧은 농담을 해줘", thread=thread)
print(response.text)

---

## 💬 대화 이어가기

이제 몇 가지 대화를 더 진행해 봅니다.

In [ ]:
response = await agent.run("로봇에 대한 짧은 농담을 해줘", thread=thread)
print(response.text)

In [ ]:
response = await agent.run("인간에 대한 짧은 농담을 해줘", thread=thread)
print(response.text)

---

## 🔍 Redis에서 채팅 기록 확인하기

### 📊 저장 구조

**Another Redis Desktop Manager**를 사용하여 Redis에 저장된 채팅 기록을 확인할 수 있습니다.

| 항목 | 설명 |
|------|------|
| **키 구조** | 각 스레드에 대해 고유한 키 생성 |
| **메시지 저장** | 해당 키 아래에 시간 순서대로 저장 |

### 📸 Redis UI 스크린샷

![Redis UI](images/maf-redis.png)